In [1]:
# import the data types
from classes.ScraperData import ScraperData
from classes.RidbData import RidbData

# import the scrapers
from classes.ReservationsScraper import ReservationsScraper
from classes.UsfsWebScraper import UsfsWebScraper

# import the storage
from classes.MysqlStorage import MysqlStorage
from classes.CsvStorage import CsvStorage

import config
import pandas as pd


Setup our storage objects

In [2]:
mysql_store = MysqlStorage()
csv_store = CsvStorage()

Setup our camping plans!

In [3]:
destination_info = dict(Latitude = config.MTHOOD_LAT, Longitude = config.MTHOOD_LON, radius=5)
start_date = '06/01/2016'
stay_length = 2

Setup scraper inputs

In [4]:
usfs_urls = pd.read_csv('data/usfs_sites.csv')
reservation_urls = pd.read_csv('data/reservation_urls.csv')
reservation_urls = reservation_urls.assign(start_date=start_date, stay_length=stay_length)

Setup objects for our data sources - RIDB API, USFS Websites, recreation.gov reservations

In [5]:
ridb_data = RidbData('ridb_merge_lab', "camping", destination_info, mysql_store)
usfs_data = ScraperData('usfs_merge_lab',usfs_urls,mysql_store,UsfsWebScraper())
res_data = ScraperData('res_merge_lab',reservation_urls,mysql_store,ReservationsScraper())

starting display
setting up web browser


Extract the data!

In [6]:
ridb_data.extract()
usfs_data.extract()
res_data.extract()

In [ ]:
ridb_data.df.shape

In [ ]:
usfs_data.df.shape

In [ ]:
res_data.df.shape

In [ ]:
ridb_data.put()
usfs_data.put()
res_data.put()

We are ready to merge!

In [ ]:
ridb_data.df.columns

In [ ]:
usfs_data.df.columns

In [ ]:
res_data.df.columns

In [13]:
from classes.DistanceMergeData import DistanceMergeData

In [36]:
merge_data = DistanceMergeData("merge_res_fs",[res_data.df, usfs_data.df],mysql_store)

In [37]:
merge_data.extract()

In [38]:
merge_data.df

,CurrentConditions,OpenSeason,Reservations,Restroom,Water,FacilityElevation,FacilityLatitude,FacilityLongitude,FacilityName,FacilityStatus,merge_index,SitesAvailable
0,None,5/20/2016 - 9/11/2016,Reserve online at www.recreation.gov or phone ...,Vault Toilets (3),Yes,2000 feet,44.68829400202167,-122.0103430470687,whispering falls campground,None,0,6
1,OPEN,April,This site can be reserved by calling Toll Free...,Vault Toilet (18),Potable Water,3200,45.50080,-121.81641,lost lake campground,Open,1,0
2,None,May 2016,This site can be reserved by calling Toll Free...,Vault Toilet,Potable Water,2000,45.073611,-121.956944,lake harriet campground,Open,2,5
3,OPEN,May,None,Vault Toilet (2),No,3000,45.45748,-121.66343,kinnikinnick (laurance lake) campground,Open,-9223372036854775808,NaN
4,No services provided.A Pack in/Pack out.,06/01,No reservations,Vault Toilet (1),No,4000,45.3505,-121.4729,fifteenmile campground,Open,-9223372036854775808,NaN
5,CLOSED,None,This site can be reserved by calling Toll Free...,Vault Toilet,Potable Water,3600,45.17945,-121.69675,clear lake campground,Closed,3,6
6,None,05/27/2016 - 09/05/2016,This site can be reserved by calling Toll Free...,Vault Toilet,Potable Water,2200,45.305236,-121.867275,camp creek campground,Closed,4,4
7,Campground not accessible due to snow,June,No reservations,Vault Toilet (1),No,4400,45.30496,-121.55537,badger lake campground,Open,-9223372036854775808,NaN
8,Eagle Creek campgroundisA reservable starting ...,May 1 - September 30,Eagle Creek Campground sites can be reserved u...,Flush toilets,Potable water available,200 - 200,45.642244,-121.925284,eagle creek campground,Open,5,2


In [42]:
merge_data.put()

Storage.put failed
(pymysql.err.OperationalError) (2013, 'Lost connection to MySQL server during query') [SQL: 'DESCRIBE `merge_res_fs`']


In [41]:
merge_data.name

'merge_res_fs'